# Notebook to access to data for the System-level Science Performance Verification Sprint February 3-5 2025

- Confluence page : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/372867091/System-level+Science+Performance+Verification+Sprint
- slides : https://docs.google.com/presentation/d/1bPjS5NGtiEL2lfjmvP1UYdG_tMEDfZxX26ayhf7MhtY/edit#slide=id.ga2f7fb1a1f_0_70

- where to find the campains
- https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

- plot Navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator/plot/%2Frepo%2Fmain/LSSTComCam%2Fruns%2FDRP%2FDP1%2Fw_2025_05%2FDM-48666/objectTableCore_coaddInputCount_SkyPlot

- Notebooks examples
- https://github.com/lsst-dm/DMTR-401/blob/main/notebooks/test_LVV-T40_T1240.ipynb
- https://github.com/lsst-dm/DMTR-412/blob/tickets/DM-38728/notebooks/test_LVV-T1751_AM1_AM2.ipynb

- author : Sylvie Dagoret-Campagne
- creattion date : 2025-02-13
- last update : 2025-02-14


- To find what I did on LSSTComCamSim : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/LightCurves/MultiColor_lightCurves-DMRehearsal2024_01-AuxTel-DZPOnCCD.ipynb
- **Confluence page** : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/443613290/Science+Verification+Sprint+Feb+2025


-* Source code : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/LightCurves/SingleBand_lightCurves_ExtractOneObject_LSSTComCamSim.ipynb

In [ ]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom

In [ ]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import random

import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default
Gaia.ROW_LIMIT = 100000
from astropy.visualization import (MinMaxInterval, AsinhStretch, ZScaleInterval, LogStretch, LinearStretch,
                                   ImageNormalize)

In [ ]:
# Set plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
from matplotlib import colors

import seaborn as sns
from itertools import cycle, islice

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 24})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})


In [ ]:
def convertVisitToDatestr(visit):

    num = visit//100_000
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return datestr

In [ ]:
def convertVisitToMJD(visit):
    return Time(convertVisitToDatestr(visit)).mjd

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666'
instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
band = "g"
#where_clause = "instrument = \'" + instrument+ "\'" + "AND band =  \'" + band + "\'" 
STARTDATE = "20241201"
where_clause = "instrument = \'" + instrument+ "\'" + "AND band =  \'" + band + "\'" +  "AND exposure.day_obs >= " + f"{STARTDATE}" 
#where="instrument='DECam' AND exposure.observation_type='science'" 
collectionStr = collection.replace("/", "_")
NDET = 9

In [ ]:
# Select the aperture radius
rap = "09"

# instrumental flux (ADU or photons)
calibFluxStr = f"apFlux{rap}instFlux"
calibFluxErrStr = f"apFlux{rap}instFluxErr"
calibFluxMagStr = f"apFlux{rap}instMag"
calibFluxMagErrStr = f"apFlux{rap}instMagErr"

# flux in nJ or Mag
calibFluxCalStr = f"apFlux{rap}calFlux"
calibFluxCalErrStr = f"apFlux{rap}calFluxErr"
calibFluxCalMagStr = f"apFlux{rap}calMag"
calibFluxCalMagErrStr = f"apFlux{rap}calMagErr"

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
for _ in registry.queryCollections():
    if "LSSTComCam/runs/DRP/DP1/w_2025_05" in _:
        print(_)

In [ ]:
if 0:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and (("Table" in datasetType.name) or ("Zeropointp" in datasetType.name) or ("fgcm" in datasetType.name) or ("transm" in datasetType.name) or ("Transm" in datasetType.name) )
            ):
                print(datasetType)

## Select the data product

In [ ]:
FLAG_ISOLATED_STAR_SOURCES = False
FLAG_SOURCES_TABLE_VISIT = True
FLAG_OBJECTS_TABLE_TRACT = True
FLAG_FGCM = False
FLAG_FGCM_CYCLE5 = True
FLAG_TRANSMISSION = True

### Object

In [ ]:
if FLAG_SOURCES_TABLE_VISIT:

    data_product = "objectTable_tract"
    datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collection,where = where_clause)
    for i, ref in enumerate(datasetRefs):
        print(i,ref.dataId)
        if i>20:
            break
    

In [ ]:
all_objectTable_tract = []
for i, ref in enumerate(datasetRefs):
    all_objectTable_tract.append(butler.get(ref))
    if i>10:
        break

### Visits

In [ ]:
#visit_table = butler.get('visitTable', dataId={'instrument': instrument}, collections = collection)   
visit_table = butler.get('ccdVisitTable', dataId={'instrument': instrument}, collections = collection)   
visit_table

In [ ]:
# Compute nightobs
visit_table["nightObs"] = visit_table.apply(lambda x: x['visitId']//100_000, axis=1)
#  compute time
visit_table["Time"] = pd.to_datetime(visit_table['obsStart'])
# get airmass for CCD Visit table
visit_table["airmass"] = visit_table["zenithDistance"].apply(lambda x: 1/np.cos(np.pi/180.*x))

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")

fig,(ax,ax2) = plt.subplots(2,1,figsize=(16,6),layout="constrained",sharex=True)
leg = ax.get_legend()
map_filtercolor = {"u":"b","g":"g","r":"r","i":"orange","z":"grey","y":"purple"}
for the_band in ["u","g","r","i","z","y"]:
    visit_table[visit_table.band == the_band].plot(x="Time",y="airmass",marker='+',c=map_filtercolor[the_band] ,lw=0.0,ax=ax,grid=True,legend=leg,label=the_band)
    visit_table[visit_table.band == the_band].plot(x="Time",y="zeroPoint",marker='+',c=map_filtercolor[the_band] ,lw=0.0,ax=ax2,grid=True,legend=leg,label=the_band)
ax.set_ylabel("airmass")
ax.set_xlabel(None)
ax2.set_ylabel("zeroPoint")

ax.xaxis.set_major_formatter(date_form)
ax2.xaxis.set_major_formatter(date_form)
ax.legend(bbox_to_anchor=(1.01, 1.02),ncols=2)
ax2.legend(bbox_to_anchor=(1.01, 1.02),ncols=2)
ax.set_title(f"{instrument} , {collectionStr}",fontsize=12)
plt.show()

### Sources

In [ ]:
# Try to get the Schema

if FLAG_SOURCES_TABLE_VISIT:

    data_product = "sourceTable_visit"
    #datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collections, where= where_clause)
    datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collection,where = where_clause)
    for i, ref in enumerate(datasetRefs):
        print(i,ref.dataId)
        if i>20:
            break
    

In [ ]:
all_sourceTable_visit = []
for i, ref in enumerate(datasetRefs):
    all_sourceTable_visit.append(butler.get(ref))
    if i>20:
            break

In [ ]:
# Extra columns to load from sourceTable_visit catalogs
names = ["psfFlux", "psfFluxErr", "psfFlux_flag", "psfFlux_apCorr", "psfFlux_apCorrErr",
         "extendedness", "detect_isPrimary", "deblend_skipped",
         "gaussianFlux", "gaussianFluxErr", "gaussianFlux_flag",
         "localPhotoCalib", "localPhotoCalibErr", "localPhotoCalib_flag"]

In [ ]:
list(all_sourceTable_visit[0].columns)

In [ ]:
df = pd.concat(all_sourceTable_visit)

In [ ]:
selection_cut = (df.parentSourceId == 0) & (df.sky_source == False) &  (df.detect_isPrimary == True) & (df.extendedness_flag == False) & (df.blendedness_flag==False)

In [ ]:
df = df[selection_cut]

In [ ]:
df

## Merge the sourceTable with the CCD visit Table

In [ ]:
df_m =  pd.merge(left=df, right=visit_table,how='left',left_on=['visit', 'detector'],right_on=['visitId', 'detector'], suffixes=('', '_vis'))

In [ ]:
df_m

In [ ]:
cmap = ListedColormap(sns.color_palette("hls", NDET))
all_det_colors = [cmap.colors[idx] for idx in range(NDET)]

In [ ]:
fig,axes = plt.subplots(3,3,figsize=(16,16),layout="constrained")

axs = axes.flatten()

for idet, ax in enumerate(axs):
    df_m[df_m.detector==idet].plot.scatter(x="x",y="y", marker=".",color=all_det_colors[idet],ax=ax,alpha=0.1)
    ax.set_aspect('equal')
    ax.set_title(f"det = {idet}")
  
    
#plt.gca().set_aspect('equal')
plt.suptitle(f"{instrument} \n {collectionStr}",fontsize=12)

## Calculate Magnitudes

In [ ]:
# Add columns into dataJoined table to have them all in one place
dataJoined["psfSn"] = dataJoined["psfFlux"]/dataJoined["psfFluxErr"]
dataJoined["psfMag"] = (dataJoined["psfFlux"].values*u.nJy).to(u.ABmag).value
dataJoined["psfMagErr"] = 2.5/np.log(10.0)*(dataJoined["psfFluxErr"].values/dataJoined["psfFlux"].values)


# This is the way to apply aperture corrections :
dataJoined[calibFluxCalStr] = dataJoined[calibFluxStr]*dataJoined["localPhotoCalib"]
dataJoined[calibFluxCalErrStr] = dataJoined[calibFluxErrStr]*dataJoined["localPhotoCalib"]


dataJoined[calibFluxCalMagStr] = (dataJoined[calibFluxCalStr].values*u.nJy).to(u.ABmag).value
dataJoined[calibFluxCalMagErrStr] = 2.5/np.log(10.0)*(dataJoined[calibFluxCalErrStr].values/dataJoined[calibFluxCalStr].values)

# NOTE: psfFlux is the fgcm calibrated flux.  I'm pretty sure you get the "instrumental" 
# flux by dividing psfFlux by the localPhotoCalib value.
dataJoined["psfInstMag"] = ((dataJoined["psfFlux"].values/dataJoined["localPhotoCalib"].values)*u.nJy).to(u.ABmag).value
dataJoined["psfGausFluxRatio"] = dataJoined["psfFlux"]/dataJoined["gaussianFlux"]

## Compute zero-point difference

In [ ]:
df_m["localPhotoCalibMag"] = (df_m["localPhotoCalib"].values*u.nJy).to(u.ABmag).value
df_m["localPhotoCalibMagErr"] =   2.5/np.log(10.0)*df_m["localPhotoCalibErr"].values/df_m["localPhotoCalib"].values
df_m["DZP"] = df_m["localPhotoCalibMag"] - df_m["zeroPoint"]
